In [2]:
import pandas as pd

states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

In [3]:
# read files as downloaded from US EIA data browser
# https://www.eia.gov/electricity/data/browser/
df_all_fuels = pd.read_csv('Net_generation_all_fuels.csv', header=4)
df_wind = pd.read_csv('Net_generation_wind.csv', header=4)

In [4]:
df_wind.head()

,Year,United States thousand megawatthours,New England thousand megawatthours,Connecticut thousand megawatthours,Maine thousand megawatthours,Massachusetts thousand megawatthours,New Hampshire thousand megawatthours,Rhode Island thousand megawatthours,Vermont thousand megawatthours,Middle Atlantic thousand megawatthours,...,New Mexico thousand megawatthours,Utah thousand megawatthours,Wyoming thousand megawatthours,Pacific Contiguous thousand megawatthours,California thousand megawatthours,Oregon thousand megawatthours,Washington thousand megawatthours,Pacific Noncontiguous thousand megawatthours,Alaska thousand megawatthours,Hawaii thousand megawatthours
0,2020,337509.81466,3917.87104,12.184,2498.41463,267.92639,524.80574,230.95429,383.586,8847.18890,...,7189.87066,802.70129,5142.78406,30521.12792,13645.32627,8549.5469,8326.25475,742.45445,163.17547,579.27897
1,2019,295882.48349,3734.04192,12.215,2493.93200,210.81692,433.40500,206.19500,377.478,7728.37600,...,6892.08700,818.68400,4162.81800,26981.21894,13735.06894,6568.8890,6677.26100,671.90093,142.59093,529.31000
2,2018,272667.45372,3556.18269,12.295,2384.29500,220.98216,406.65900,158.60553,373.346,7587.84599,...,6091.55100,794.55500,4057.15800,29371.48680,14023.95080,7447.4420,7900.09400,756.94900,155.01500,601.93400
3,2017,254302.69522,3444.08146,12.706,2332.85300,232.62246,411.59200,148.90200,305.406,7748.96300,...,4594.57108,858.25200,4320.65500,25974.69658,12822.94833,6226.5930,6925.15525,673.74200,141.68700,532.05500
4,2016,226992.56213,2646.00615,12.742,1667.10300,216.12315,432.32400,26.53200,291.182,7437.26700,...,3605.42100,822.28200,4389.33800,28708.01369,13509.03869,7157.1280,8041.84700,808.42800,169.30100,639.12700


In [5]:
# define useful function to unpivot data into long form, clean column names, and select only state data
def cleaner(df, fuel):
    df = pd.melt(df, id_vars=['Year'], var_name='State')
    df['State'] = df.State.str.replace(' thousand megawatthours', '')
    df['Fuel']=fuel
    df = df.loc[df.State.isin(states)]
    
    return df

In [8]:
# clean both wind and all fuels dataframes
dfleft = cleaner(df_all_fuels,'All')
dfright = cleaner(df_wind, 'Wind')

# join dataframes and clean up column names
df2 = dfleft.merge(dfright, on=['Year','State'], how='left')
df2 = df2.rename(columns={'value_x':'All GWh', 'value_y':'Wind GWh'})
df2 = df2.drop(['Fuel_x', 'Fuel_y'], axis=1)

# some states did not produce any wind energy, fill NAs with 0
df2 = df2.fillna(0)

In [9]:
df2.tail()

,Year,State,All GWh,Wind GWh
995,2005,Hawaii,11522.80535,6.632
996,2004,Hawaii,11410.40259,7.495
997,2003,Hawaii,10976.37100,1.572
998,2002,Hawaii,11663.07000,1.614
999,2001,Hawaii,10633.09300,2.122


In [31]:
df2.to_csv('windAllData2.csv')